In [2]:
from keras.models import Sequential, load_model, save_model
from keras.layers import Dense, Dropout, LeakyReLU
from keras.optimizers import *
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

validation_part, test_part = 0.2, 0.2
epoches_num = 200
batch_size = 64

np.random.seed = 99
ZERO_LABEL, POS_LABEL = 'BENIGN', 'PortScan'

Using Theano backend.


In [3]:
def shuffle_data(x, y):
    '''Shuffles data inplace'''
    indeces = np.random.shuffle(np.arange(x.shape[0]))
    x[:], y[:] = x[indeces], y[indeces]

def scale(data):
    '''Scales numpy array, deletes nan, inf'''
    inf, not_zero_ind = np.isinf(data), np.nonzero(data)[0]
    data = np.nan_to_num(data)
    data[inf] = data[np.logical_not(inf)].max()
    data[data == 0] = 0 if not_zero_ind.size == 0 else data[not_zero_ind].mean()
    return StandardScaler().fit_transform(data) / 2 + 0.5

def gen(train=True, data=None):
    if data is None:
        global X_train, y_train, X_val, y_val
    else:
        X_train, y_train, X_val, y_val = data
    x, y = (X_train, y_train) if train else (X_val, y_val)
    while True:
        shuffle_data(x, y)
        for i in range(x.shape[0] // batch_size):
            yield x[i * batch_size: (i+1) * batch_size], y[i * batch_size: (i+1) * batch_size]

In [4]:
def str_arr_to_one_hot(arr, max_one_hot_len=200):
    temp_arr = LabelEncoder().fit_transform(arr)
    one_hot_len = len(set(temp_arr))
    return (True, to_categorical(temp_arr)) if one_hot_len < max_one_hot_len else (False, one_hot_len)

def data_to_one_hot(data):
    ans, not_used = [], []
    for column in data.columns:
        data_arr = data[column].as_matrix()
        if type(data_arr[0]) is str and column != 'Flow Bytes/s':
            is_used, data_arr = str_arr_to_one_hot(data_arr)
            if not is_used:
                not_used.append([column, data_arr])
                continue
        else:
            data_arr = scale(data_arr.astype(np.float64).reshape(-1, 1))
        ans.append(data_arr)
    if not_used:
        print("Didn't use columns:")
        print(not_used)
    return np.concatenate(ans, axis=1)

def load_data(filename):
    '''Returns (x, y) if labels are in the file, else (x, IDnums)'''
    data = pd.read_csv(filename, index_col='IDNum')
    x_one_hot = data_to_one_hot(data[ [c for c in data.columns if c != 'Label'] ])
    if 'Label' in data.columns:
        y_one_hot = data_to_one_hot(pd.DataFrame(data['Label']))        
        fir_elem_lab = data['Label'].as_matrix()[0]
        if (fir_elem_lab == ZERO_LABEL and y_one_hot[0, 1] == 1) or \
                (fir_elem_lab == POS_LABEL and y_one_hot[0, 0] == 1):
            y_one_hot = y_one_hot[:, ::-1]
        return x_one_hot, y_one_hot
    return x_one_hot, np.array(data.index)

In [4]:
X, Y = load_data('FWHA-PortScan-Train.csv')
shuffle_data( X, Y )
val_part = int(validation_part * X.shape[0])
X_train, X_val, y_train, y_val = X[val_part:], X[:val_part], Y[val_part:], Y[:val_part]

Didn't use columns:
[['Flow ID', 168626], ['Source IP', 3425], ['Destination IP', 4244], ['Flow Packets/s', 56389]]


In [5]:
model = Sequential()
model.add(Dense( 2000, input_dim=X.shape[1] )) #, activation='relu'
model.add(Dropout(0.4))
model.add(LeakyReLU())

model.add(Dense(1000))
model.add(Dropout(0.4))
model.add(LeakyReLU())

model.add(Dense(500))
model.add(Dropout(0.5))
model.add(LeakyReLU())

model.add(Dense(2, activation='softmax'))

#opt = Adadelta(lr=0.01)
opt = SGD(lr=0.0001, nesterov=True)
#opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2000)              460000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000)              0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               500500    
__________

In [6]:
model.fit_generator(gen(),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=epoches_num,
                    validation_data=gen(train=False),
                    validation_steps=X_val.shape[0] // batch_size,
                    callbacks=[ModelCheckpoint("compet_models_minor/mlp_{epoch:d}-{acc:.2f}.hdf5", save_best_only=True)])

Epoch 1/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.5793 - acc: 0.7152 - val_loss: 0.6325 - val_acc: 0.7318
Epoch 2/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.5017 - acc: 0.7859 - val_loss: 0.5416 - val_acc: 0.7748
Epoch 3/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.4305 - acc: 0.8419 - val_loss: 0.4524 - val_acc: 0.8067
Epoch 4/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.3652 - acc: 0.8839 - val_loss: 0.3744 - val_acc: 0.8359
Epoch 5/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.3088 - acc: 0.9121 - val_loss: 0.3148 - val_acc: 0.8607
Epoch 6/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.2625 - acc: 0.9315 - val_loss: 0.2726 - val_acc: 0.8763
Epoch 7/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.2255 - acc: 0.9444 - val_loss: 0.2404 - val_acc: 0.8972
Epoch 8/200
2599/2599 [=========================

Epoch 61/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0309 - acc: 0.9925 - val_loss: 0.0332 - val_acc: 0.9896
Epoch 62/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0303 - acc: 0.9926 - val_loss: 0.0325 - val_acc: 0.9899
Epoch 63/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0303 - acc: 0.9927 - val_loss: 0.0320 - val_acc: 0.9900
Epoch 64/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0306 - acc: 0.9927 - val_loss: 0.0315 - val_acc: 0.9901
Epoch 65/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0297 - acc: 0.9927 - val_loss: 0.0313 - val_acc: 0.9901
Epoch 66/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0297 - acc: 0.9929 - val_loss: 0.0310 - val_acc: 0.9902
Epoch 67/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0292 - acc: 0.9930 - val_loss: 0.0306 - val_acc: 0.9902
Epoch 68/200
2599/2599 [=================

2599/2599 [==============================] - 19s 7ms/step - loss: 0.0173 - acc: 0.9964 - val_loss: 0.0146 - val_acc: 0.9958
Epoch 180/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0177 - acc: 0.9963 - val_loss: 0.0144 - val_acc: 0.9960
Epoch 181/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0175 - acc: 0.9965 - val_loss: 0.0145 - val_acc: 0.9959
Epoch 182/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0172 - acc: 0.9964 - val_loss: 0.0144 - val_acc: 0.9960
Epoch 183/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0168 - acc: 0.9966 - val_loss: 0.0143 - val_acc: 0.9961
Epoch 184/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0173 - acc: 0.9964 - val_loss: 0.0143 - val_acc: 0.9961
Epoch 185/200
2599/2599 [==============================] - 19s 7ms/step - loss: 0.0174 - acc: 0.9964 - val_loss: 0.0143 - val_acc: 0.9961
Epoch 186/200
2599/2599 [=======================

StopIteration: 

In [5]:
X_test, IDnums = load_data('FWHA-PortScan-Test.csv')
model = load_model('mlp_198-1.00.hdf5')
y_test = np.argmax(model.predict(X_test), axis=1)

Didn't use columns:
[['Flow ID', 229683], ['Source IP', 3667], ['Destination IP', 4401], ['Flow Packets/s', 68847]]


In [6]:
answers = np.vstack((IDnums, [POS_LABEL * y + ZERO_LABEL * (1-y) for y in y_test])).T


In [7]:
answers

array([['170367', 'BENIGN'],
       ['170368', 'BENIGN'],
       ['170369', 'BENIGN'],
       ..., 
       ['456831', 'BENIGN'],
       ['456832', 'BENIGN'],
       ['456833', 'BENIGN']], 
      dtype='<U21')

In [ ]:
np.savetxt('answers_2.csv', answers, fmt='%s,%s', header='IDNum,Label', comments='')